In [4]:
import pandas as pd
import numpy as np
import nltk as nl
from nltk.sentiment import SentimentIntensityAnalyzer

df = pd.read_csv('data/bolsonaro_tweets.csv')

ModuleNotFoundError: No module named 'nltk'

In [ ]:
df.head()

In [ ]:
# Verificando se há valores nulos
print(df.isnull().sum())
# Verificando o tipo de dado de cada coluna
print(df.dtypes)
# Descrição do dataset
print(df.shape)

In [ ]:
# Converter a coluna 'date' para o tipo datetime
df['date'] = pd.to_datetime(df['date'])
# Preencher com a data anterior
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].fillna(method='ffill') # forward fill

print(df.isnull().sum())
